# Sectors Review

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [2]:
data = pd.read_csv('../../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740.0,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,"3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US",...,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08,2024-06-12,2024-05-23
AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142.0,NYSE,USD,USA,MANUFACTURING,HOUSEHOLD APPLIANCES,"11270 WEST PARK PLACE, MILWAUKEE, WI, US",...,3.248,15.19,1.141,92.44,63.34,83.92,81.37,1.207840e+08,2024-08-15,2024-07-31
ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"100 ABBOTT PARK ROAD, ABBOTT PARK, IL, US",...,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09,2024-08-15,2024-07-15
ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"1 NORTH WAUKEGAN ROAD, NORTH CHICAGO, IL, US",...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09,2024-08-15,2024-07-15
ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373.0,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",...,3.041,18.21,1.227,384.33,277.47,301.44,329.36,6.263840e+08,2024-08-15,2024-07-11


In [3]:
data.drop(['AssetType',
           'CIK',
           'Exchange',
           'Description',
           'Currency',
           'Country',
           'Address',
           'DividendPerShare',
           'FiscalYearEnd',
           'LatestQuarter',
           'DividendDate',
           'ExDividendDate',
          ], inplace=True, axis=1)

In [4]:
data.replace('None', np.nan, inplace=True)

In [5]:
data['MarketCapitalization'] = data['MarketCapitalization'].astype(float)
data['Beta'] = data['Beta'].astype(float)
data['PEGRatio'] = data['PEGRatio'].astype(float)
data['PERatio'] = data['PERatio'].astype(float)
data['BookValue'] = data['BookValue'].astype(float)
data['DividendYield'] = data['DividendYield'].astype(float)
data['EPS'] = data['EPS'].astype(float)
data['RevenuePerShareTTM'] = data['RevenuePerShareTTM'].astype(float)
data['ProfitMargin'] = data['ProfitMargin'].astype(float)
data['OperatingMarginTTM'] = data['OperatingMarginTTM'].astype(float)
data['ReturnOnAssetsTTM'] = data['ReturnOnAssetsTTM'].astype(float)
data['EBITDA'] = data['EBITDA'].astype(float)
data['RevenueTTM'] = data['RevenueTTM'].astype(float)
data['GrossProfitTTM'] = data['GrossProfitTTM'].astype(float)
data['DilutedEPSTTM'] = data['DilutedEPSTTM'].astype(float)
data['QuarterlyEarningsGrowthYOY'] = data['QuarterlyEarningsGrowthYOY'].astype(float)
data['QuarterlyRevenueGrowthYOY'] = data['QuarterlyRevenueGrowthYOY'].astype(float)

In [6]:
data['Sector'].unique()

array(['LIFE SCIENCES', 'MANUFACTURING', 'TRADE & SERVICES', 'TECHNOLOGY',
       'ENERGY & TRANSPORTATION', 'FINANCE', 'REAL ESTATE & CONSTRUCTION'],
      dtype=object)

In [7]:
data.groupby(['Sector']).count()['Name'].to_frame().sort_values('Name', ascending=False)

,Name
Sector,
MANUFACTURING,96
LIFE SCIENCES,80
ENERGY & TRANSPORTATION,76
TRADE & SERVICES,75
TECHNOLOGY,69
FINANCE,65
REAL ESTATE & CONSTRUCTION,42


In [10]:
multps = {
    'capital'  : ['MarketCapitalization', 'EBITDA', 'BookValue', 'RevenueTTM', 'GrossProfitTTM', ],
    'ratio'    : ['PERatio', 'PEGRatio',],
    'dividend' : ['DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'DilutedEPSTTM',],
    'margin'   : ['ProfitMargin', 'OperatingMarginTTM',],
    'returns'  : ['ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY'],
}

## LIFE SCIENCES

In [8]:
lf = data[data['Sector']=='LIFE SCIENCES'].copy()
lf.drop('Sector', axis=1, inplace=True)

In [9]:
lf.head()

,Name,Industry,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendYield,EPS,RevenuePerShareTTM,...,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,3M Company,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,5.808631e+10,8.146000e+09,NaN,1.905,8.790,0.0267,-12.73,58.89,...,1.546,10.51,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08
ABT,Abbott Laboratories,PHARMACEUTICAL PREPARATIONS,1.804344e+11,1.048500e+10,32.72,5.990,22.310,0.0212,3.17,23.41,...,4.72,4.904,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09
ABBV,AbbVie Inc,PHARMACEUTICAL PREPARATIONS,3.065727e+11,2.611600e+10,51.52,0.449,4.535,0.0357,3.37,30.77,...,5.29,27.74,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09
A,Agilent Technologies Inc,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,3.892675e+10,1.612000e+09,31.62,2.701,21.310,0.0071,4.22,22.53,...,6.12,6.66,6.24,24.88,1.049,155.06,96.30,136.10,133.04,2.917610e+08
APD,Air Products and Chemicals Inc,INDUSTRIAL INORGANIC CHEMICALS,5.525416e+10,4.115100e+09,22.45,1.372,67.080,0.0269,11.07,54.64,...,4.143,3.461,4.989,13.79,0.822,299.60,209.25,263.88,257.75,2.223060e+08


In [20]:
lf[['Name']+multps['capital']].sort_values('MarketCapitalization', ascending=False).head(10)

,Name,MarketCapitalization,EBITDA,BookValue,RevenueTTM,GrossProfitTTM
Symbol,,,,,,
LLY,Eli Lilly and Company,7.243399e+11,1.337370e+10,14.230,3.593210e+10,2.191160e+10
PG,Procter & Gamble Company,3.962049e+11,2.288500e+10,21.450,8.403900e+10,3.924600e+10
JNJ,Johnson & Johnson,3.869879e+11,3.118100e+10,29.720,8.657600e+10,6.395400e+10
ABBV,AbbVie Inc,3.065727e+11,2.611600e+10,4.535,5.440300e+10,4.152700e+10
MRK,Merck & Company Inc,2.917797e+11,3.305500e+10,15.940,6.248000e+10,4.207700e+10
TMO,Thermo Fisher Scientific Inc,2.292129e+11,1.081900e+10,124.170,4.234600e+10,1.899800e+10
LIN,Linde plc Ordinary Shares,2.182451e+11,1.220200e+10,80.780,3.276100e+10,1.391400e+10
DHR,Danaher Corporation,1.978214e+11,7.460000e+09,68.920,2.356800e+10,1.894900e+10
ABT,Abbott Laboratories,1.804344e+11,1.048500e+10,22.310,4.072500e+10,2.457900e+10


In [22]:
lf[['Name']+multps['capital']].sort_values('EBITDA', ascending=False).head(10)

,Name,MarketCapitalization,EBITDA,BookValue,RevenueTTM,GrossProfitTTM
Symbol,,,,,,
MRK,Merck & Company Inc,2.917797e+11,3.305500e+10,15.940,6.248000e+10,4.207700e+10
JNJ,Johnson & Johnson,3.869879e+11,3.118100e+10,29.720,8.657600e+10,6.395400e+10
ABBV,AbbVie Inc,3.065727e+11,2.611600e+10,4.535,5.440300e+10,4.152700e+10
PG,Procter & Gamble Company,3.962049e+11,2.288500e+10,21.450,8.403900e+10,3.924600e+10
BMY,Bristol-Myers Squibb Company,9.053028e+10,1.817200e+10,8.130,4.553400e+10,3.637800e+10
LLY,Eli Lilly and Company,7.243399e+11,1.337370e+10,14.230,3.593210e+10,2.191160e+10
HCA,HCA Holdings Inc,9.369376e+10,1.335600e+10,-6.170,6.834700e+10,2.317700e+10
GILD,Gilead Sciences Inc,9.723860e+10,1.266500e+10,14.080,2.745000e+10,2.162400e+10
LIN,Linde plc Ordinary Shares,2.182451e+11,1.220200e+10,80.780,3.276100e+10,1.391400e+10


In [21]:
lf[['Name']+multps['capital']].sort_values('BookValue', ascending=False).head(10)

,Name,MarketCapitalization,EBITDA,BookValue,RevenueTTM,GrossProfitTTM
Symbol,,,,,,
BIO,Bio-Rad Laboratories Inc,8.760474e+09,5.029910e+08,317.32,2.605238e+09,1.568430e+09
REGN,Regeneron Pharmaceuticals Inc,1.180926e+11,4.519100e+09,261.41,1.348910e+10,7.020000e+09
ROP,Roper Technologies Inc. Common Stock,5.523429e+10,2.667400e+09,169.21,6.574400e+09,3.752800e+09
TMO,Thermo Fisher Scientific Inc,2.292129e+11,1.081900e+10,124.17,4.234600e+10,1.899800e+10
BIIB,Biogen Inc,3.290929e+10,2.519800e+09,104.48,9.663100e+09,7.895100e+09
UHS,Universal Health Services Inc,1.457016e+10,2.012872e+09,96.97,1.501751e+10,5.162775e+09
TFX,Teleflex Incorporated,1.083702e+10,6.305570e+08,96.75,3.007838e+09,1.550087e+09
LH,Laboratory Corporation of America Holdings,1.981534e+10,1.655800e+09,95.53,1.248760e+10,4.385100e+09
BDX,Becton Dickinson and Company,6.859268e+10,5.070000e+09,88.74,1.971600e+10,8.823000e+09


In [14]:
lf.groupby('Industry').agg({'Name': 'count',
                            'MarketCapitalization': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                           })

Name MarketCapitalization  \
                                                   count                  sum   
Industry                                                                        
AGRICULTURAL CHEMICALS                                 2         2.223604e+10   
AGRICULTURAL PRODUCTION-CROPS                          1         3.860550e+10   
AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMM...     1         7.950805e+10   
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)        5         3.544207e+11   
CHEMICALS & ALLIED PRODUCTS                            1         8.127291e+09   
ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS          1         1.045178e+11   
IN VITRO & IN VIVO DIAGNOSTIC SUBSTANCES               1         3.932146e+10   
INDUSTRIAL INORGANIC CHEMICALS                         2         2.734992e+11   
INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY...     5         3.442476e+11   
INDUSTRIAL ORGANIC CHEMICALS                           2         5.597436e+10   
INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY &...     3         8.519576e+10   
LABORATORY ANALYTICAL INSTRUMENTS                      4         7.335886e+10   
MEASURING & CONTROLLING DEVICES, NEC                   3         2.702312e+11   
OPHTHALMIC GOODS                                       1         1.855998e+10   
OPTICAL INSTRUMENTS & LENSES                           1         9.373232e+10   
ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES & ...     5         2.622998e+11   
PAINTS, VARNISHES, LACQUERS, ENAMELS & ALLIED P...     1         2.785369e+10   
PERFUMES, COSMETICS & OTHER TOILET PREPARATIONS        3         1.533251e+11   
PHARMACEUTICAL PREPARATIONS                           12         2.497098e+12   
PLASTIC MATERIAL, SYNTH RESIN/RUBBER, CELLULOS ...     1         1.495222e+10   
PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN ELA...     4         9.341702e+10   
SERVICES-GENERAL MEDICAL & SURGICAL HOSPITALS, NEC     2         1.082639e+11   
SERVICES-MEDICAL LABORATORIES                          2         3.604424e+10   
SERVICES-MISC HEALTH & ALLIED SERVICES, NEC            1         1.205121e+10   
SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUMES...     3         4.910279e+11   
SPECIALTY CLEANING, POLISHING AND SANITATION PR...     1         1.671695e+10   
SURGICAL & MEDICAL INSTRUMENTS & APPARATUS            10         4.753832e+11   
X-RAY APPARATUS & TUBES & RELATED IRRADIATION A...     2         5.612110e+10   

                                                                         \
                                                             min idxmin   
Industry                                                                  
AGRICULTURAL CHEMICALS                              8.799740e+09    MOS   
AGRICULTURAL PRODUCTION-CROPS                       3.860550e+10   CTVA   
AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMM...  7.950805e+10     TT   
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)     1.223017e+10   TECH   
CHEMICALS & ALLIED PRODUCTS                         8.127291e+09    FMC   
ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS       1.045178e+11    MDT   
IN VITRO & IN VIVO DIAGNOSTIC SUBSTANCES            3.932146e+10   IDXX   
INDUSTRIAL INORGANIC CHEMICALS                      5.525416e+10    APD   
INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY...  2.494786e+10    FTV   
INDUSTRIAL ORGANIC CHEMICALS                        2.523889e+10    IFF   
INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY &...  2.005107e+10    TER   
LABORATORY ANALYTICAL INSTRUMENTS                   8.760474e+09    BIO   
MEASURING & CONTROLLING DEVICES, NEC                1.261334e+10   TRMB   
OPHTHALMIC GOODS                                    1.855998e+10    COO   
OPTICAL INSTRUMENTS & LENSES                        9.373232e+10   KLAC   
ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES & ...  1.876848e+10   ALGN   
PAINTS, VARNISHES, LACQUERS, ENAMELS & ALLIED P...  2.785369e+10    PPG   
PERFUMES, COSMETICS & OTHER TOILET PREPA

In [23]:
summ = lf.groupby('Industry').agg({'Name': 'count',
                            'MarketCapitalization': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'EBITDA': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'BookValue': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'DividendYield': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'EPS': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'RevenuePerShareTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'ProfitMargin': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'OperatingMarginTTM': ['min', 'idxmin', 'max', 'idxmax'],
                            'ReturnOnAssetsTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'ReturnOnEquityTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'RevenueTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'QuarterlyEarningsGrowthYOY': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'QuarterlyRevenueGrowthYOY': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                           })
summ_pv = pd.DataFrame(columns=summ.index, index=summ.columns, data=summ.to_numpy().transpose())
summ_pv

Industry                         AGRICULTURAL CHEMICALS  \
Name                      count                       2   
MarketCapitalization      sum             22236045000.0   
                          min              8799740000.0   
                          idxmin                    MOS   
                          max             13436305000.0   
...                                                 ...   
QuarterlyRevenueGrowthYOY sum                    -0.526   
                          min                    -0.269   
                          idxmin                     CF   
                          max                    -0.257   
                          idxmax                    MOS   

Industry                         AGRICULTURAL PRODUCTION-CROPS  \
Name                      count                              1   
MarketCapitalization      sum                    38605500000.0   
                          min                    38605500000.0   
                          idxmin                          CTVA   
                          max                    38605500000.0   
...                                                        ...   
QuarterlyRevenueGrowthYOY sum                            -0.08   
                          min                            -0.08   
                          idxmin                          CTVA   
                          max                            -0.08   
                          idxmax                          CTVA   

Industry                         AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMML ENVIRONMENTS  \
Name                      count                                                   1              
MarketCapitalization      sum                                         79508054000.0              
                          min                                         79508054000.0              
                          idxmin                                                 TT              
                          max                                         79508054000.0              
...                                                                             ...              
QuarterlyRevenueGrowthYOY sum                                                 0.128              
                          min                                                 0.128              
                          idxmin                                                 TT              
                          max                                                 0.128              
                          idxmax                                                 TT              

Industry                         BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)  \
Name                      count                                                5   
MarketCapitalization      sum                                     354420660000.0   
                          min                                      12230172000.0   
                          idxmin                                            TECH   
                          max                                     178761597000.0   
...                                                                          ...   
QuarterlyRevenueGrowthYOY sum                                             -0.064   
                          min                                             -0.299   
                          idxmin                                            MRNA   
                          max                                               0.22   
                          idxmax                                            AMGN   

Industry                         CHEMICALS & ALLIED PRODUCTS  \
Name                      count                            1   
MarketCapitalization      sum                   8127291000.0   
                          min                   8127291000.0   
                          idxmin                        

In [27]:
summ_pv.loc['MarketCapitalization']

Industry,AGRICULTURAL CHEMICALS,AGRICULTURAL PRODUCTION-CROPS,AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMML ENVIRONMENTS,"BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)",CHEMICALS & ALLIED PRODUCTS,ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS,IN VITRO & IN VIVO DIAGNOSTIC SUBSTANCES,INDUSTRIAL INORGANIC CHEMICALS,"INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY, AND CONTROL",INDUSTRIAL ORGANIC CHEMICALS,...,PHARMACEUTICAL PREPARATIONS,"PLASTIC MATERIAL, SYNTH RESIN/RUBBER, CELLULOS (NO GLASS)","PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN ELASTOMERS","SERVICES-GENERAL MEDICAL & SURGICAL HOSPITALS, NEC",SERVICES-MEDICAL LABORATORIES,"SERVICES-MISC HEALTH & ALLIED SERVICES, NEC","SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUMES, COSMETICS","SPECIALTY CLEANING, POLISHING AND SANITATION PREPARATIONS",SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,X-RAY APPARATUS & TUBES & RELATED IRRADIATION APPARATUS
sum,22236045000.0,38605500000.0,79508054000.0,354420660000.0,8127291000.0,104517829000.0,39321465000.0,273499226000.0,344247650000.0,55974355000.0,...,2497097840000.0,14952218000.0,93417015000.0,108263920000.0,36044243000.0,12051214000.0,491027862000.0,16716946000.0,475383245000.0,56121099000.0
min,8799740000.0,38605500000.0,79508054000.0,12230172000.0,8127291000.0,104517829000.0,39321465000.0,55254155000.0,24947855000.0,25238893000.0,...,10555658000.0,14952218000.0,10997001000.0,14570156000.0,16228905000.0,12051214000.0,24789742000.0,16716946000.0,10784012000.0,18955719000.0
idxmin,MOS,CTVA,TT,TECH,FMC,MDT,IDXX,APD,FTV,IFF,...,CTLT,CE,ALB,UHS,DGX,DVA,CHD,CLX,SOLV,HOLX
max,13436305000.0,38605500000.0,79508054000.0,178761597000.0,8127291000.0,104517829000.0,39321465000.0,218245071000.0,197821366000.0,30735462000.0,...,724339851000.0,14952218000.0,37050687000.0,93693764000.0,19815338000.0,12051214000.0,396204933000.0,16716946000.0,124226642000.0,37165380000.0
idxmax,CF,CTVA,TT,AMGN,FMC,MDT,IDXX,LIN,DHR,LYB,...,LLY,CE,DOW,HCA,LH,DVA,PG,CLX,SYK,GEHC
